# OPER 618 Game Theory and Math Programming
## Homework #3 - Computing Concepts for Normal Form Games
## Hosley, Brandon

In [1]:
import pyomo.environ as pyo
import itertools

1. **Maxmin Strategies**. Consider the two-player normal form game depicted in Figure 4.1 of
the textbook (and shown below), as well as the file *"OPER 618 - Homework 3 - Problem 1 -
Sets and Parameters.py"* posted on Canvas.

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|              |       | Player 2 |       |
|:------------:|:-----:|:--------:|:-----:|
|              |       |   $k_1$  | $k_2$ |
|              | $j_1$ |   0, 1   |  6, 0 |
| **Player 1** | $j_2$ |   2, 0   |  5, 2 |
|              | $j_3$ |   3, 4   |  3, 3 |

</div>

<ol type="a">
  <li>
    For this non-zero sum game, use Python/Pyomo to formulate and solve the problem of
    finding the <i>maxmin</i> strategy for <b>Player 1</b>, allowing for mixed strategies. Discuss the
    significance of the optimal solution
  </li>
</ol>


In [2]:
def problem1_a():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2,3}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
    [0, 0, 6], 
    [0, 2, 5],
    [0, 3, 3]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
    m.p1 = pyo.Var(m.J,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]

    # Model  
    m.objfnvalue = pyo.Objective(expr = m.U1, sense = pyo.maximize)
    m.MincriteriaP1=pyo.ConstraintList()
    for k in m.K:
        m.MincriteriaP1.add(sum(u1[j][k]*m.p1[j] for j in m.J) >= m.U1)
    
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(m.p1[j] for j in m.J)==1)
                        
    # Solve model
    pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1))

    for j in m.J:
        print('p(',j,') = %.3f' % pyo.value(m.p1[j]))

problem1_a()

U1 = 3.000
p( 1 ) = 0.000
p( 2 ) = 0.000
p( 3 ) = 1.000


This solution shows that Player 1 will always choose $j_3$ under the maxmin strategy. Unsuprising, as they are gauranteed to have a utility no less than 3.

<ol type="a" start="2">
  <li>
    For this non-zero sum game, use Python/Pyomo to formulate and solve the problem of
    finding the <i>maxmin</i> strategy for <b>Player 2</b>, allowing for mixed strategies. Discuss the
    significance of the optimal solution.
  </li>
</ol>

In [3]:
def problem1_b():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2,3}
    m.K = {1,2}

    # Input parameter data
    u2=[[0,0,0],
    [0, 1, 0], 
    [0, 0, 2],
    [0, 4, 3]]

    # Decision Variables
    m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
    m.p2 = pyo.Var(m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]

    # Model  
    m.objfnvalue = pyo.Objective(expr = m.U2, sense = pyo.maximize)
    m.MincriteriaP2=pyo.ConstraintList()
    for j in m.J:
        m.MincriteriaP2.add(sum(u2[j][k]*m.p2[k] for k in m.K) >= m.U2)
    
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(m.p2[k] for k in m.K)==1)

    # Solve model
    pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U2 = %.3f' % pyo.value(m.U2)) 

    for k in m.K:
        print('p(',k,') = %.3f' % pyo.value(m.p2[k]))

problem1_b()

U2 = 0.667
p( 1 ) = 0.667
p( 2 ) = 0.333


Because both of Player 2's choices have equal minimums and neither is (weakly) dominant, a pure strategy does not exist.
As a result pyomo returns a solution in which the minimum non-zero expected results are equal. 

2. **Discriminating among Nash Equilibria with the LCP formulation**. Consider the two-player
normal form game depicted in Figure 4.1 of the textbook (and shown below), as well as the
file *"OPER 618 - Homework 3 - Problem 2 - LCP System of Equations.py"* posted on Canvas.

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|              |       | Player 2 |       |
|:------------:|:-----:|:--------:|:-----:|
|              |       |   $k_1$  | $k_2$ |
|              | $j_1$ |   0, 1   |  6, 0 |
| **Player 1** | $j_2$ |   2, 0   |  5, 2 |
|              | $j_3$ |   3, 4   |  3, 3 |

</div>

As formulated, the Python/Pyomo Concrete Model finds the Nash equilibrium $s = (s_1, s_2) = \left((0.67,0.33,0), (0.33, 0.67)\right)$ 
with expected payoffs of $(u_1, u_2) = (4, 0.67)$.
However, this is only one of the three Nash equilibria, and it doesn't seem to be a very
desirable one... at least not for Player 2.

<ol type="a">
  <li>
    Modify and solve the Python/Pyomo formulation to find the Nash equilibrium that
    <u>minimizes the maximum difference in the players' utilities</u>.
  </li>
</ol>

In [25]:
def problem2_a():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2,3}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
    [0, 0, 6], 
    [0, 2, 5],
    [0, 3, 3]]
    u2=[[0,0,0],
    [0, 1, 0], 
    [0, 0, 2],
    [0, 4, 3]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.Reals) # Player 1's utility
    m.U2 = pyo.Var(domain=pyo.Reals) # Player 2's utility
    m.s1 = pyo.Var(m.J,domain=pyo.NonNegativeReals) # s1[j] is the probabilty that Player 1 plays action j
    m.r1 = pyo.Var(m.J,domain=pyo.NonNegativeReals) # Player 1's slack variable associated with the utility of playing action j 
    m.s2 = pyo.Var(m.K,domain=pyo.NonNegativeReals) # s2[k] is the probabilty that Player 2 plays action k
    m.r2 = pyo.Var(m.K,domain=pyo.NonNegativeReals) # Player 2's slack variable associated with the utility of playing action k 
    m.z = pyo.Var(domain=pyo.NonNegativeReals) # A dummy variable to serve as the objective function
    
    # Model (Formulation from page 91 of the text)
    m.objfnvalue = pyo.Objective(expr = m.U2, sense = pyo.maximize)
    m.UtilityForPlayer1=pyo.ConstraintList()
    for j in m.J:
        m.UtilityForPlayer1.add(sum(u1[j][k]*m.s2[k] for k in m.K)  == m.U1)
    m.UtilityForPlayer2=pyo.ConstraintList()
    for k in m.K:
        m.UtilityForPlayer2.add(sum(u2[j][k]*m.s1[j] for j in m.J) == m.U2)

    m.ProbabilityDistributionForP1 = pyo.Constraint(expr = sum(m.s1[j] for j in m.J) == 1)   
    m.ProbabilityDistributionForP2 = pyo.Constraint(expr = sum(m.s2[k] for k in m.K) == 1)                         
    m.ComplementarySlacknessForPlayer1=pyo.ConstraintList()
    for j in m.J:
        m.ComplementarySlacknessForPlayer1.add(m.s1[j]*m.r1[j] == 0)
    m.ComplementarySlacknessForPlayer2=pyo.ConstraintList()
    for k in m.K:
        m.ComplementarySlacknessForPlayer2.add(m.s2[k]*m.r2[k] == 0)

    # Solve model
    results=pyo.SolverFactory('ipopt').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1)) 
    for j in m.J:
        print('s1(',j,') = %.3f' % pyo.value(m.s1[j]))
    print('U2 = %.3f' % pyo.value(m.U2))    
    for k in m.K:
        print('s2(',k,') = %.3f' % pyo.value(m.s2[k]))

problem2_a()

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Ipopt 3.14.12\x3a Converged to a locally infeasible
      point. Problem may be infeasible.
U1 = 3.000
s1( 1 ) = 0.004
s1( 2 ) = 0.333
s1( 3 ) = 0.662
U2 = 2.654
s2( 1 ) = 0.500
s2( 2 ) = 0.500


<ol type="a" start="2">
  <li>
    Modify and solve the Python/Pyomo formulation with an objective function that will
    find the remaining Nash equilibrium. Discuss the merits of your approach as it pertains
    to generalizability for other two-player games.
  </li>
</ol>

For some reason this program was particularly difficult, and never seemed to behave as expected. A third Nash Equilibrium was apparent by inspection, however, giving some guidance on the expected answer. With this method I have attempted to utilize pyomo to find the optimal pure strategy, but in the process of exploring possible answers, the player two Utility often behaved counter-intuitively depending on the solver used.

After an irresponsible amount of time, the closest I could get was on in which the strategy declarations are correct, but the reported utility is not. This approach is not likely to be robust, nor would I want to use it as is for any other problem.

In [5]:
def problem2_b():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2,3}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
    [0, 0, 6], 
    [0, 2, 5],
    [0, 3, 3]]
    u2=[[0,0,0],
    [0, 1, 0], 
    [0, 0, 2],
    [0, 4, 3]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.Reals) # Player 1's utility
    m.U2 = pyo.Var(domain=pyo.Reals) # Player 2's utility
    m.s1 = pyo.Var(m.J,domain=pyo.Binary) # s1[j] is the probabilty that Player 1 plays action j
    m.s2 = pyo.Var(m.K,domain=pyo.Binary) # s2[k] is the probabilty that Player 2 plays action k

    # Model (Formulation from page 91 of the text)
    m.objfnvalue = pyo.Objective(expr = m.U2-m.U1, sense = pyo.maximize)
    m.UtilityForPlayer1=pyo.ConstraintList()
    for j in m.J:
        m.UtilityForPlayer1.add(sum(u1[j][k]*m.s2[k] for k in m.K) <= m.U1)

    m.UtilityForPlayer2=pyo.ConstraintList()
    for k in m.K:
        m.UtilityForPlayer2.add(sum(u2[j][k]*m.s1[j] for j in m.J) >= m.U2)

    m.ProbabilityDistributionForP1 = pyo.Constraint(expr = sum(m.s1[j] for j in m.J) == 1)   
    m.ProbabilityDistributionForP2 = pyo.Constraint(expr = sum(m.s2[k] for k in m.K) == 1)                         

    # Solve model
    pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1)) 
    for j in m.J:
        print('s1(',j,') = %.3f' % pyo.value(m.s1[j]))
    print('U2 = %.3f' % pyo.value(m.U2))    
    for k in m.K:
        print('s2(',k,') = %.3f' % pyo.value(m.s2[k]))

problem2_b()

U1 = 3.000
s1( 1 ) = 0.000
s1( 2 ) = 0.000
s1( 3 ) = 1.000
U2 = 3.000
s2( 1 ) = 1.000
s2( 2 ) = 0.000


<ol type="a" start="3">
  <li>
    Discuss a method one could implement to find <u>all</u> Nash equilibria using the LCP formulation.
  </li>
</ol>

An effective method to find all of the nash Equilibria using the LCP formulation would be to step through the same steps as the Lemke-Howson Algorithm. In the case of implementation within Pyomo, I believe that one method to do so would be to manually set parts of the $s$ vector to values corresponding to what may be representative of the $G\times G$. In this particular case, ons should have been able to set a value of a strictly dominant strategy for any individual player as 1, but perhaps doing so interferes with the interior point method of ipopt. 

### Normal form game used for Problems 3, 4, & 5. 
Consider the two-player normal form game depicted below, which is a modified form of "Chicken".

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|              |       | Player 2 |       |
|:------------:|:-----:|:--------:|:-----:|
|              |       |   $k_1$  | $k_2$ |
| **Player 1** | $j_1$ |   0, 0   |  9, 3 |
|              | $j_2$ |   3, 9   |  7, 7 |

</div>
The following figure (Duffy and Feltovich, 2010) shows the possible payoffs for Player 1 and
Player 2, wherein the respective regions of feasible payoffs, correlated equilibrium payoffs,
and Nash equilibrium payoffs are depicted with the indicated letter(s).

![](https://www.mathworks.com/matlabcentral/mlc-downloads/downloads/e568cfd1-4a80-11e4-9553-005056977bd0/66a2877e-5aac-4a1d-b090-5035e35db9b2/images/screenshot.gif)

3. **Correlated Equilibria and Nash Equilibria**. Consider the Python/Pyomo models in the following files, which correspond to the two functional representations in Section 4.6.
    - *OPER 618 - Homework 3 - Problem 3 - Correlated Equilibrium (LP) - Chicken.py*
    - *OPER 618 - Homework 3 - Problem 3 - Nash Equilibrium (NLP) - Chicken.py*

    Modify the objective function(s) and resolve the model(s) as necessary to identify (a) the
    four correlated equilibria corresponding to the extreme points of the region of correlated
    equilibria payoff pairs from the diagram and (b) the three correlated equilibria
    corresponding to the three Nash equilibria corresponding to the extreme points of the
    region of NE payoff pairs from the diagram.

For the lower-middle correlated equilibrium point, we simply use the code as graciously provided by Dr. Lunday.

In [6]:
def CorrelatedEquilibriumLow():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
        [0, 0, 9], 
        [0, 3, 7]]
    u2=[[0,0,0],
        [0, 0, 3], 
        [0, 9, 7]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
    m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
    m.p = pyo.Var(m.J,m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]
    
    # Model  
    m.objfnvalue = pyo.Objective(expr = m.U1+m.U2, sense = pyo.minimize)
    m.UtilityForPlayer1= pyo.Constraint(expr= m.U1==sum(sum(m.p[j,k]*u1[j][k] for j in m.J) for k in m.K))
    m.UtilityForPlayer2= pyo.Constraint(expr= m.U2==sum(sum(m.p[j,k]*u2[j][k] for j in m.J) for k in m.K))
    m.NEcriteriaP1=pyo.ConstraintList()
    for j in m.J:
        for jp in m.J:
            if jp!=j:
                m.NEcriteriaP1.add(sum(m.p[j,k]*u1[j][k] for k in m.K) >= sum(m.p[j,k]*u1[jp][k] for k in m.K))
    m.NEcriteriaP2=pyo.ConstraintList()
    for k in m.K:
        for kp in m.K:
            if kp!=k:
                m.NEcriteriaP2.add(sum(m.p[j,k]*u2[j][k] for j in m.J) >= sum(m.p[j,k]*u2[j][kp] for j in m.J))
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(sum(m.p[j,k] for j in m.J) for k in m.K)==1)
                        
    # Solve model
    results=pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1))   
    print('U2 = %.3f' % pyo.value(m.U2)) 

    for j in m.J:
        for k in m.K:
            print('p(',j,',',k,') = %.3f' % pyo.value(m.p[j,k]))

CorrelatedEquilibriumLow()

U1 = 4.500
U2 = 4.500
p( 1 , 1 ) = 0.250
p( 1 , 2 ) = 0.375
p( 2 , 1 ) = 0.375
p( 2 , 2 ) = 0.000


For the upper-middle correlated equilibrium point, we simply change the objective function to maximize.

In [7]:
def CorrelatedEquilibriumHigh():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
        [0, 0, 9], 
        [0, 3, 7]]
    u2=[[0,0,0],
        [0, 0, 3], 
        [0, 9, 7]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
    m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
    m.p = pyo.Var(m.J,m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]
    
    # Model  
    m.objfnvalue = pyo.Objective(expr = m.U1+m.U2, sense = pyo.maximize)
    m.UtilityForPlayer1= pyo.Constraint(expr= m.U1==sum(sum(m.p[j,k]*u1[j][k] for j in m.J) for k in m.K))
    m.UtilityForPlayer2= pyo.Constraint(expr= m.U2==sum(sum(m.p[j,k]*u2[j][k] for j in m.J) for k in m.K))
    m.NEcriteriaP1=pyo.ConstraintList()
    for j in m.J:
        for jp in m.J:
            if jp!=j:
                m.NEcriteriaP1.add(sum(m.p[j,k]*u1[j][k] for k in m.K) >= sum(m.p[j,k]*u1[jp][k] for k in m.K))
    m.NEcriteriaP2=pyo.ConstraintList()
    for k in m.K:
        for kp in m.K:
            if kp!=k:
                m.NEcriteriaP2.add(sum(m.p[j,k]*u2[j][k] for j in m.J) >= sum(m.p[j,k]*u2[j][kp] for j in m.J))
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(sum(m.p[j,k] for j in m.J) for k in m.K)==1)
                        
    # Solve model
    results=pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1))   
    print('U2 = %.3f' % pyo.value(m.U2)) 

    for j in m.J:
        for k in m.K:
            print('p(',j,',',k,') = %.3f' % pyo.value(m.p[j,k]))

CorrelatedEquilibriumHigh()

U1 = 6.429
U2 = 6.429
p( 1 , 1 ) = 0.000
p( 1 , 2 ) = 0.286
p( 2 , 1 ) = 0.286
p( 2 , 2 ) = 0.429


For the middle Nash equilibrium point we once again use the free code.

In [8]:
def NashEquilibrium():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
        [0, 0, 9], 
        [0, 3, 7]]
    u2=[[0,0,0],
        [0, 0, 3], 
        [0, 9, 7]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
    m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
    m.p1 = pyo.Var(m.J,domain=pyo.NonNegativeReals) # p1[j] is the probabilty that Player 1 plays action j
    m.p2 = pyo.Var(m.K,domain=pyo.NonNegativeReals) # p2[k] is the probabilty that Player 2 plays action k
    m.p = pyo.Var(m.J,m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]

    # Model  
    m.objfnvalue = pyo.Objective(expr = 0*m.U1+m.U2, sense = pyo.minimize)
    m.UtilityForPlayer1= pyo.Constraint(expr= m.U1==sum(sum(m.p[j,k]*u1[j][k] for j in m.J) for k in m.K))
    m.UtilityForPlayer2= pyo.Constraint(expr= m.U2==sum(sum(m.p[j,k]*u2[j][k] for j in m.J) for k in m.K))
    m.NEcriteriaP1=pyo.ConstraintList()
    for j in m.J:
        for jp in m.J:
            if jp!=j:
                m.NEcriteriaP1.add(sum(m.p[j,k]*u1[j][k] for k in m.K) >= sum(m.p[j,k]*u1[jp][k] for k in m.K))
    m.NEcriteriaP2=pyo.ConstraintList()
    for k in m.K:
        for kp in m.K:
            if kp!=k:
                m.NEcriteriaP2.add(sum(m.p[j,k]*u2[j][k] for j in m.J) >= sum(m.p[j,k]*u2[j][kp] for j in m.J))
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(sum(m.p[j,k] for j in m.J) for k in m.K)==1)
    m.JointPDFCalculations=pyo.ConstraintList()
    for j in m.J:
        for k in m.K:
            m.JointPDFCalculations.add(m.p[j,k]==m.p1[j]*m.p2[k])
    m.ProbabilityDistributionForP1 = pyo.Constraint(expr = sum(m.p1[j] for j in m.J) == 1)   
    m.ProbabilityDistributionForP2 = pyo.Constraint(expr = sum(m.p2[k] for k in m.K) == 1)   
                        
    # Solve model
    results=pyo.SolverFactory('ipopt').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1)) 
    for j in m.J:
        print('p1(',j,') = %.3f' % pyo.value(m.p1[j]))
        
    print('U2 = %.3f' % pyo.value(m.U2)) 
    for k in m.K:
        print('p2(',k,') = %.3f' % pyo.value(m.p2[k]))

    for j in m.J:
        for k in m.K:
            print('p(',j,',',k,') = %.3f' % pyo.value(m.p[j,k]))

NashEquilibrium()

U1 = 5.400
p1( 1 ) = 0.400
p1( 2 ) = 0.600
U2 = 5.400
p2( 1 ) = 0.400
p2( 2 ) = 0.600
p( 1 , 1 ) = 0.160
p( 1 , 2 ) = 0.240
p( 2 , 1 ) = 0.240
p( 2 , 2 ) = 0.360


For the asymmetric, player-favored points we keep seeking the maximization utility, but subtract the utility of the opposite player.

In [9]:
def FavoredEquilibrium(player=1):
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
        [0, 0, 9], 
        [0, 3, 7]]
    u2=[[0,0,0],
        [0, 0, 3], 
        [0, 9, 7]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
    m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
    m.p = pyo.Var(m.J,m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]
    
    # Model
    m.objfnvalue = pyo.Objective(expr = m.U1-m.U2 if player == 1 else m.U2-m.U1, sense = pyo.maximize)
    m.UtilityForPlayer1= pyo.Constraint(expr= m.U1==sum(sum(m.p[j,k]*u1[j][k] for j in m.J) for k in m.K))
    m.UtilityForPlayer2= pyo.Constraint(expr= m.U2==sum(sum(m.p[j,k]*u2[j][k] for j in m.J) for k in m.K))
    m.NEcriteriaP1=pyo.ConstraintList()
    for j in m.J:
        for jp in m.J:
            if jp!=j:
                m.NEcriteriaP1.add(sum(m.p[j,k]*u1[j][k] for k in m.K) >= sum(m.p[j,k]*u1[jp][k] for k in m.K))
    m.NEcriteriaP2=pyo.ConstraintList()
    for k in m.K:
        for kp in m.K:
            if kp!=k:
                m.NEcriteriaP2.add(sum(m.p[j,k]*u2[j][k] for j in m.J) >= sum(m.p[j,k]*u2[j][kp] for j in m.J))
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(sum(m.p[j,k] for j in m.J) for k in m.K)==1)
                        
    # Solve model
    results=pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1))   
    print('U2 = %.3f' % pyo.value(m.U2)) 

    for j in m.J:
        for k in m.K:
            print('p(',j,',',k,') = %.3f' % pyo.value(m.p[j,k]))

FavoredEquilibrium(player=1)
FavoredEquilibrium(player=2)

U1 = 9.000
U2 = 3.000
p( 1 , 1 ) = 0.000
p( 1 , 2 ) = 1.000
p( 2 , 1 ) = 0.000
p( 2 , 2 ) = 0.000
U1 = 3.000
U2 = 9.000
p( 1 , 1 ) = 0.000
p( 1 , 2 ) = -0.000
p( 2 , 1 ) = 1.000
p( 2 , 2 ) = 0.000


4. **The Region of Correlated Equilibria**. Identify a correlated equilibrium yielding a pair of
payoffs in the region $B_l$ (i.e., the region of correlated equilibria that are not Nash equilibria
and have lower-than-NE payoffs), as well as a correlated equilibrium yielding a pair of
payoffs in the region $B_h$.

By using the previous correlated equilibrium extreme point functions and symmetrically weighting the criteria values. The weight, $w$ below, is a value between 0 and 1 which represents the proportion of distance between the 'lower correlated point' and the 'middle Nash' point that the new value is. 

In [10]:
def CorrelatedEquilibriumB_l():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
        [0, 0, 9], 
        [0, 3, 7]]
    u2=[[0,0,0],
        [0, 0, 3], 
        [0, 9, 7]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
    m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
    m.p = pyo.Var(m.J,m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]

    # Weight
    w = 0.2 # Must be between 0 and 1
    
    # Model  
    m.objfnvalue = pyo.Objective(expr = m.U1+m.U2, sense = pyo.minimize)
    m.UtilityForPlayer1= pyo.Constraint(expr= m.U1==sum(sum(m.p[j,k]*u1[j][k] for j in m.J) for k in m.K))
    m.UtilityForPlayer2= pyo.Constraint(expr= m.U2==sum(sum(m.p[j,k]*u2[j][k] for j in m.J) for k in m.K))
    m.NEcriteriaP1=pyo.ConstraintList()
    for j in m.J:
        for jp in m.J:
            if jp!=j:
                m.NEcriteriaP1.add(sum(m.p[j,k]*u1[j][k] for k in m.K) >= sum(m.p[j,k]*u1[jp][k] for k in m.K)+w)
    m.NEcriteriaP2=pyo.ConstraintList()
    for k in m.K:
        for kp in m.K:
            if kp!=k:
                m.NEcriteriaP2.add(sum(m.p[j,k]*u2[j][k] for j in m.J) >= sum(m.p[j,k]*u2[j][kp] for j in m.J)+w)
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(sum(m.p[j,k] for j in m.J) for k in m.K)==1)
                        
    # Solve model
    results=pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1))   
    print('U2 = %.3f' % pyo.value(m.U2)) 

    for j in m.J:
        for k in m.K:
            print('p(',j,',',k,') = %.3f' % pyo.value(m.p[j,k]))

CorrelatedEquilibriumB_l()

U1 = 4.800
U2 = 4.800
p( 1 , 1 ) = 0.200
p( 1 , 2 ) = 0.400
p( 2 , 1 ) = 0.400
p( 2 , 2 ) = 0.000


THe upper region operates in the same way as above, with the restriction on the weight value and it representing a proportion of distance from the Nash Equilibrium.

In [11]:
def CorrelatedEquilibriumB_h():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
        [0, 0, 9], 
        [0, 3, 7]]
    u2=[[0,0,0],
        [0, 0, 3], 
        [0, 9, 7]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
    m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
    m.p = pyo.Var(m.J,m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]
    
    # Weight
    w = 0.33 # Must be between 0 and 1

    # Model  
    m.objfnvalue = pyo.Objective(expr = m.U1+m.U2, sense = pyo.maximize)
    m.UtilityForPlayer1= pyo.Constraint(expr= m.U1==sum(sum(m.p[j,k]*u1[j][k] for j in m.J) for k in m.K))
    m.UtilityForPlayer2= pyo.Constraint(expr= m.U2==sum(sum(m.p[j,k]*u2[j][k] for j in m.J) for k in m.K))
    m.NEcriteriaP1=pyo.ConstraintList()
    for j in m.J:
        for jp in m.J:
            if jp!=j:
                m.NEcriteriaP1.add(sum(m.p[j,k]*u1[j][k] for k in m.K) >= sum(m.p[j,k]*u1[jp][k] for k in m.K)+w)
    m.NEcriteriaP2=pyo.ConstraintList()
    for k in m.K:
        for kp in m.K:
            if kp!=k:
                m.NEcriteriaP2.add(sum(m.p[j,k]*u2[j][k] for j in m.J) >= sum(m.p[j,k]*u2[j][kp] for j in m.J)+w)
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(sum(m.p[j,k] for j in m.J) for k in m.K)==1)
                        
    # Solve model
    results=pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1))   
    print('U2 = %.3f' % pyo.value(m.U2)) 

    for j in m.J:
        for k in m.K:
            print('p(',j,',',k,') = %.3f' % pyo.value(m.p[j,k]))

CorrelatedEquilibriumB_h()

U1 = 6.334
U2 = 6.334
p( 1 , 1 ) = 0.000
p( 1 , 2 ) = 0.333
p( 2 , 1 ) = 0.333
p( 2 , 2 ) = 0.334


5. **Unstable Solutions**. Show that the solution corresponding to the joint payoffs of $(7,7)$ is
neither a correlated equilibrium nor a Nash equilibrium.

While this situation can be seen intuitively from the payoff matrix, it may also be seen by the modifications to the correlated equilibrium function that we have been using up to this point. The $(7,7)$ value is achievable, but only when the equilibrium criteria are both switched from $\geq$ to $\leq$, which effectively represents that the player is playing in an altruistic manner, seeking maximum utility for their opponent. While this outcome may be desirable, it does break the assumption of Homo economicus players.

In [12]:
def FragileCooperation():
    m = pyo.ConcreteModel()

    # Input sets
    m.J = {1,2}
    m.K = {1,2}

    # Input parameter data
    u1=[[0,0,0],
        [0, 0, 9], 
        [0, 3, 7]]
    u2=[[0,0,0],
        [0, 0, 3], 
        [0, 9, 7]]

    # Decision Variables
    m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
    m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
    m.p = pyo.Var(m.J,m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]
    
    # Model  
    m.objfnvalue = pyo.Objective(expr = m.U1+m.U2, sense = pyo.maximize)
    m.UtilityForPlayer1= pyo.Constraint(expr= m.U1==sum(sum(m.p[j,k]*u1[j][k] for j in m.J) for k in m.K))
    m.UtilityForPlayer2= pyo.Constraint(expr= m.U2==sum(sum(m.p[j,k]*u2[j][k] for j in m.J) for k in m.K))
    m.NEcriteriaP1=pyo.ConstraintList()
    for j in m.J:
        for jp in m.J:
            if jp!=j:
                m.NEcriteriaP1.add(sum(m.p[j,k]*u1[j][k] for k in m.K) <= sum(m.p[j,k]*u1[jp][k] for k in m.K))
    m.NEcriteriaP2=pyo.ConstraintList()
    for k in m.K:
        for kp in m.K:
            if kp!=k:
                m.NEcriteriaP2.add(sum(m.p[j,k]*u2[j][k] for j in m.J) <= sum(m.p[j,k]*u2[j][kp] for j in m.J))
    m.ProbabilityDistribution= pyo.Constraint(expr= sum(sum(m.p[j,k] for j in m.J) for k in m.K)==1)
                        
    # Solve model
    results=pyo.SolverFactory('glpk').solve(m,tee=False)

    # Output results
    print('U1 = %.3f' % pyo.value(m.U1))   
    print('U2 = %.3f' % pyo.value(m.U2)) 

    for j in m.J:
        for k in m.K:
            print('p(',j,',',k,') = %.3f' % pyo.value(m.p[j,k]))

FragileCooperation()

U1 = 7.000
U2 = 7.000
p( 1 , 1 ) = 0.000
p( 1 , 2 ) = 0.000
p( 2 , 1 ) = 0.000
p( 2 , 2 ) = 1.000
